In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import torchvision.models as models
import cv2


In [3]:
models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /datasets/home/55/455/yut107/.torch/models/alexnet-owt-4df8aa71.pth
100%|██████████| 244418560/244418560 [00:16<00:00, 14797680.34it/s]


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [2]:
class Extractor(nn.Module):
    def __init__(self):
        super(Extractor, self).__init__()
        
        self.net = models.alexnet(pretrained=True)
        self.net.classifier = nn.Sequential(
            *(self.net.classifier[i] for i in range(6)))
        
    def forward(self, images): 
        return self.net(images)

In [3]:
def read_img(img_path):
    img = plt.imread(img_path)
    img = cv2.resize(img,(224,224))
    img = img / 255
    img = img.transpose((2,0,1))
    return img
    

In [4]:
with open("./food-101/meta/train.txt","r") as f:
    training_list = f.readlines()
with open("./food-101/meta/test.txt","r") as f:
    test_list = f.readlines()
    
with open('./food-101/meta/labels.txt','r') as f:
    labels = f.readlines()
    
training_list = ["./food-101/images/"+ path[:-1] +".jpg" for path in training_list]
test_list = ["./food-101/images/"+ path[:-1] +".jpg" for path in test_list]
labels = [label[:-1].replace(" ",'_').lower() for label in labels]

In [5]:
net = Extractor()
net.cuda()

Extractor(
  (net): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5)
      (1): Linear(in_features=9216, out_features=4096, bias=True)
      (2): ReLU(inplace)
      (3): 

In [8]:
import pickle

test_data = []
counter = 0
for i in range(len(test_list)):
    img = read_img(test_list[i])
    x = torch.FloatTensor(img).view(1,3,224,224)
    x = Variable(x.cuda())
    feature = net(x)
    label_id = labels.index(test_list[i].split("/")[-2])
    test_data.append((label_id,feature.cpu().data.numpy()))
    if i % 1000 == 0:
        print(i)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


In [9]:
 with open("test_data_alexnet.pkl","wb") as f:
        pickle.dump(test_data, f)

In [10]:
len(test_data)

25250

In [6]:
import pickle

training_data = []
counter = 0
for i in range(len(training_list)):
    img = read_img(training_list[i])
    x = torch.FloatTensor(img).view(1,3,224,224)
    x = Variable(x.cuda())
    feature = net(x)
    label_id = labels.index(training_list[i].split("/")[-2])
    training_data.append((label_id,feature.cpu().data.numpy()))
    if i % 1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000


In [7]:
len(training_data)

75747

In [8]:
 with open("training_data_alexnet.pkl","wb") as f:
        pickle.dump(training_data, f)

In [29]:
training_data[-1]

(100, array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32))